# Исследование рынка книг (SQL)

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
        'pwd': 'Sdf4$2;d-d30pp', # пароль
        'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
        'port': 6432, # порт подключения
        'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
                                                         db_config['pwd'], 
                                                         db_config['host'], 
                                                         db_config['port'],
                                                         db_config['db']) 
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Выведем первые строки каждой таблицы

In [2]:
books = ''' 
SELECT 
    * 
FROM 
    books 
---LIMIT 5
'''
pd.io.sql.read_sql(books, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


In [3]:
authors = ''' 
SELECT 
    * 
FROM 
    authors 

'''
pd.io.sql.read_sql(authors, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


In [4]:
publishers = ''' 
SELECT 
    * 
FROM 
    publishers 

'''
pd.io.sql.read_sql(publishers, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


In [5]:
ratings = ''' 
SELECT 
    * 
FROM 
    ratings 

'''
pd.io.sql.read_sql(ratings, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


In [6]:
reviews = ''' 
SELECT 
    * 
FROM 
    reviews 

'''
pd.io.sql.read_sql(reviews, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


Всего в датасете books представлено 1000 книг, в authors 636 авторов, в publishers 340 издателей, в ratings 6456 оценок и в reviews 2793 ревью от пользователей.

## Посчитаем, сколько книг вышло после 1 января 2000 года

In [7]:
books_count = ''' 
SELECT 
    COUNT(*) AS cnt 
FROM 
    books 
WHERE
    publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(books_count, con = engine)

,cnt
0,819


После 1 января 2000 года вышло 819 книг, что является подавляющим большинством из нашей библиотеки.

## Посчитаем для каждой книги количество обзоров и среднюю оценку

In [61]:
books_rat_rew = ''' 
SELECT 
    title,
    COUNT(DISTINCT review_id) AS reviews_cnt,
    AVG(rating) AS avg_rating 
FROM 
    books 
    LEFT OUTER JOIN reviews on reviews.book_id = books.book_id
    INNER JOIN ratings on ratings.book_id = books.book_id
GROUP BY
    title
ORDER BY
    reviews_cnt DESC
'''
pd.io.sql.read_sql(books_rat_rew, con = engine)

,title,reviews_cnt,avg_rating
0,Memoirs of a Geisha,8,4.138462
1,Twilight (Twilight #1),7,3.662500
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,The Glass Castle,6,4.206897
4,Eat Pray Love,6,3.395833
...,...,...,...
994,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
995,The Natural Way to Draw,0,3.000000
996,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
997,Essential Tales and Poems,0,4.000000


993 книги имеют обзоры (7 книг из нашего датасета их не имеют), лидером по их количеству является - Memoirs of a Geisha.

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц;

In [9]:
books_publisher = ''' 
SELECT 
    publisher,
    COUNT(title) AS book_cnt
FROM 
    books 
    INNER JOIN publishers on publishers.publisher_id = books.publisher_id
WHERE
    num_pages > 50
GROUP BY
    publisher
ORDER BY
    book_cnt DESC
LIMIT 1
'''
pd.io.sql.read_sql(books_publisher, con = engine)

,publisher,book_cnt
0,Penguin Books,42


Лидером среди издательств по количеству книг (за вычетом брошюр) является - Penguin Books, с 42 книгами.

## Определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

In [10]:
books_authors = ''' 
SELECT 
    authors.author,
    AVG(ratings.rating) AS average,
    COUNT(ratings.rating) AS rating_cnt
FROM 
    books 
    INNER JOIN authors on authors.author_id = books.author_id
    INNER JOIN ratings on ratings.book_id = books.book_id

GROUP BY
    authors.author
HAVING
    COUNT(ratings.rating) > 50
ORDER BY
    average DESC
LIMIT 5
'''
pd.io.sql.read_sql(books_authors, con = engine)

,author,average,rating_cnt
0,J.K. Rowling/Mary GrandPré,4.288462,312
1,Agatha Christie,4.283019,53
2,Markus Zusak/Cao Xuân Việt Khương,4.264151,53
3,J.R.R. Tolkien,4.240964,166
4,Roald Dahl/Quentin Blake,4.209677,62


Самая высокая оценка у автора Гарри Поттера, Д. Роулинг. Самое большое количество оценка также принадлежит ей.

## Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [85]:
ratings_reviews = ''' 
SELECT 
    AVG(SUB.count_review) AS avg_count_review
FROM (
    SELECT
        username,
        COUNT(text) AS count_review
    FROM
        reviews
    GROUP BY
        username) 
    AS Sub 
    INNER JOIN (SELECT
        username,
        COUNT(ratings) AS count_ratings
    FROM
        ratings
    GROUP BY
        username
    HAVING
        COUNT(ratings) > 50
    ORDER BY
        count_ratings DESC) AS Sub2 on Sub2.username = Sub.username
'''
pd.io.sql.read_sql(ratings_reviews, con = engine)

,avg_count_review
0,24.333333


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24,3. Тот кто часто стаит оценки, чаще пишет обзоры.